In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a=pd.read_csv('/content/drive/MyDrive/train (1).csv')
b=pd.read_csv('/content/drive/MyDrive/test (1).csv')

In [ ]:
train=pd.DataFrame(a)
test=pd.DataFrame(b)

In [ ]:
df = pd.concat([train,test], ignore_index=True)

In [ ]:
import pickle
dt=pickle.load(open('/content/drive/MyDrive/dt_model.sav','rb'))

In [ ]:
lr=pickle.load(open('/content/drive/MyDrive/LR_model.sav','rb'))

In [ ]:
svm=pickle.load(open('/content/drive/MyDrive/SVM_modell.sav','rb'))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
def create_cnn_model():
  meta_model = Sequential()
  meta_model.add(Dense(64, activation='relu', input_shape=(3,)))
  meta_model.add(Dropout(0.2))
  meta_model.add(Dense(1, activation='sigmoid'))
  meta_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return meta_model

In [ ]:
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
stacked_predictions = []
true_labels = []

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [ ]:
def modeltrain(X_train):
  svm_pred = svm.predict(X_train)
  dt_pred = dt.predict(X_train)
  lr_pred = lr.predict(X_train)
  s=np.column_stack((svm_pred, dt_pred,lr_pred))
  return s

In [ ]:
def modeltest(X_train):
  svm_pred = svm.predict(X_train)
  dt_pred = dt.predict(X_train)
  lr_pred = lr.predict(X_train)
  s=np.column_stack((svm_pred, dt_pred,lr_pred))
  return s

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
accuracy_scores=[]
for train_index, val_index in skf.split(X, y):
  X_train, X_val=[],[]
  y_train, y_val =[],[]
  for i in (train_index):
    X_train.append(X.iloc[i])
    y_train.append([y.iloc[i]])
  y_train=np.array(y_train)
  for i in val_index:
    X_val.append(X.iloc[i])
    y_val.append([y.iloc[i]])
  y_val=np.array(y_val)
  meta_X_train=modeltrain(X_train)
  meta_model = create_cnn_model()
  meta_model.fit(meta_X_train, y_train, epochs=10, batch_size=32)
  meta_X_val=modeltest(X_val)
  predictions = (meta_model.predict(meta_X_val) > 0.5).astype("int32")
  accuracy=accuracy_score(predictions,y_val)
  accuracy_scores.append(accuracy)
  print(accuracy)

Epoch 1/10
206/206 [==============================] - 1s 2ms/step - loss: 0.4484 - accuracy: 0.9755
Epoch 2/10
206/206 [==============================] - 0s 2ms/step - loss: 0.1261 - accuracy: 0.9837
Epoch 3/10
206/206 [==============================] - 0s 2ms/step - loss: 0.0659 - accuracy: 0.9842
Epoch 4/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0571 - accuracy: 0.9840
Epoch 5/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0554 - accuracy: 0.9846
Epoch 6/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0551 - accuracy: 0.9849
Epoch 7/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0543 - accuracy: 0.9851
Epoch 8/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0556 - accuracy: 0.9848
Epoch 9/10
206/206 [==============================] - 1s 3ms/step - loss: 0.0546 - accuracy: 0.9846
Epoch 10/10
52/52 [==============================] - 0s 1ms/step
0.9908759124087592
Epoch 1/10
206/2

In [ ]:
final_acc=sum(accuracy_scores)/len(accuracy_scores)
final_acc*100


98.57606479157319

In [ ]:
accuracy_scores

[0.9908759124087592,
 0.9854014598540146,
 0.9829580036518564,
 0.9860012172854534,
 0.9835666463785758]

In [ ]:
import tensorflow as tf
meta_model.save('cnnkmodel.h5')